In [1]:
if'automol'in globals(): import importlib; importlib.reload(automol)

In [2]:
from automol.pipeline import Pipeline

In [3]:
config_yaml = 'qm9_dataset_test_config.yaml'
pipeline = Pipeline(config_yaml)
pipeline.print_spec()

/home/ching/miniconda3/envs/automol/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'regression': {'RandomForestRegressor': <class 'sklearn.ensemble._forest.RandomForestRegressor'>, 'GradientBoostingRegressor': <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>, 'GaussianProcessRegressor': <class 'sklearn.gaussian_process._gpr.GaussianProcessRegressor'>, 'LinearRegression': <class 'sklearn.linear_model._base.LinearRegression'>, 'MLPRegressor': <class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'>}, 'classification': {'GaussianProcessClassifier': <class 'sklearn.gaussian_process._gpc.GaussianProcessClassifier'>, 'SGDClassifier': <class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>, 'MLPClassifier': <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>}}
amount: 10
dataset_class: QM9
dataset_location: lsdf:///lsdf/kit/iti/projects/aimat-mlflow/datasets/dsgdb9nsd
labels:
- homo
models_filter:
- w
- - GaussianProcessRegressor
  - MLPRegressor
  - LinearRegression
  - GradientBoostingRegressor
problem: regression



In [4]:
pipeline.spec['labels']

['homo']

In [5]:
pipeline.data_set.data['homo']

0   -0.2348
1   -0.2581
2   -0.2313
3   -0.2113
4   -0.2697
5   -0.2949
6   -0.2258
7   -0.2156
8   -0.2228
9   -0.2238
Name: homo, dtype: float64

In [6]:
pipeline.data_set.data

,atom_count,tag,index,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,smiles
0,16,gdb,79782.0,3.01826,2.13202,1.86767,1.8508,64.90,-0.2348,0.0213,0.2561,791.1727,0.127306,-437.784490,-437.778794,-437.777850,-437.814266,24.622,OC1C2C3OC4C1N2C34
1,12,gdb,5049.0,3.59797,2.48091,1.74436,3.4141,63.44,-0.2581,-0.1055,0.1525,719.9173,0.086042,-373.696465,-373.690429,-373.689485,-373.726613,22.475,N=c1nccncn1
2,19,gdb,72010.0,2.47439,1.94603,1.44610,3.1663,78.86,-0.2313,-0.0027,0.2286,969.6103,0.160943,-385.883282,-385.876028,-385.875084,-385.914622,29.715,CC12CC3CC1C(=O)C32
3,13,gdb,130188.0,3.57797,1.68018,1.14520,1.3202,65.86,-0.2113,-0.0155,0.1957,963.1908,0.093247,-448.936999,-448.930233,-448.929289,-448.968004,25.861,Nc1n[nH]c2ncoc12
4,16,gdb,19995.0,3.74954,1.39769,1.13593,4.7096,70.23,-0.2697,0.0167,0.2864,1049.0618,0.131290,-342.753830,-342.746245,-342.745301,-342.786361,27.847,N#CC1NC1C1CC1
5,12,gdb,53722.0,3.54666,0.68916,0.64840,3.7533,74.59,-0.2949,-0.1255,0.1694,1670.6324,0.073926,-435.479548,-435.470149,-435.469205,-435.515981,30.366,CC(=O)C#CC(=O)C#N
6,18,gdb,75220.0,3.80464,1.14855,1.02987,1.1958,85.85,-0.2258,0.0133,0.2390,1213.0853,0.147119,-364.769640,-364.761233,-364.760289,-364.802255,32.775,C#CC12CC=C(C)C1N2
7,22,gdb,84018.0,2.16940,1.62268,1.41278,1.1438,82.09,-0.2156,0.0850,0.3005,1095.5024,0.195044,-404.327291,-404.318431,-404.317487,-404.360106,34.621,CN1C2CCOCC21C
8,16,gdb,70139.0,2.49927,2.01226,1.28078,3.6451,72.46,-0.2228,-0.0097,0.2131,958.1591,0.126390,-434.075696,-434.068322,-434.067378,-434.107186,29.566,N=C1NC=NC2CC12O
9,21,gdb,59637.0,3.32904,1.01945,0.83090,1.1542,82.31,-0.2238,0.0227,0.2465,1441.5229,0.180662,-424.238328,-424.228450,-424.227506,-424.273148,36.347,CC1=CC(C(C)O)OC1


In [7]:
pipeline.train()

/home/ching/miniconda3/envs/automol/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model 'GaussianProcessRegressor' with feature 'fingerprint' has MAE: 0.220733
Model 'GaussianProcessRegressor' with feature 'fingerprint' has MSE: 0.048737
Model 'GaussianProcessRegressor' with feature 'fingerprint' has R2S: -3651.805796
Model 'GaussianProcessRegressor' with feature 'rdkit' has MAE: 0.220733
Model 'GaussianProcessRegressor' with feature 'rdkit' has MSE: 0.048737
Model 'GaussianProcessRegressor' with feature 'rdkit' has R2S: -3651.805796
Model 'MLPRegressor' with feature 'fingerprint' has MAE: 0.047233
Model 'MLPRegressor' with feature 'fingerprint' has MSE: 0.002254
Model 'MLPRegressor' with feature 'fingerprint' has R2S: -167.961372
Model 'MLPRegressor' with feature 'rdkit' has MAE: 1.941577
Model 'MLPRegressor' with feature 'rdkit' has MSE: 6.616130
Model 'MLPRegressor' with feature 'rdkit' has R2S: -495878.160072
Model 'LinearRegression' with feature 'fingerprint' has MAE: 0.034017
Model 'LinearRegression' with feature 'fingerprint' has MSE: 0.001167
Model 'LinearRe